In [52]:
# Build a formula evaluator

In [53]:
import pickle

def load_pickled_object(file_path):
    with open(file_path, 'rb') as file:
        obj = pickle.load(file)
    return obj

In [54]:
generic_formula_dictionary = load_pickled_object('generic_formula_dictionary.pkl')
series_dict = load_pickled_object('series_dict.pkl')

In [55]:
series_dict["Sheet1"]

[Series(series_id=SeriesId(sheet_name='Sheet1', series_header='col_1', series_header_cell_row=2, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='col_1', formulas=[None, None], values=[1, 3], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=2, row=3, coordinate='B3', sheet_name=None, value=None, value_type=None, formula=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>),
 Series(series_id=SeriesId(sheet_name='Sheet1', series_header='col_2', series_header_cell_row=2, series_header_cell_column=3), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='col_2', formulas=[None, None], values=[2, 4], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=3, row=3, coordinate='C3', sheet_name=None, value=None, value_type=None, formula=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>)

In [56]:
import pandas as pd
import xlcalculator
import ast
import copy
import xlcalculator.ast_nodes as ast_nodes
import xlcalculator.tokenizer as tokenizer



class FormulaEvaluator:

    def __init__(self, series_dict):
        self.local_env = {"SUM": self.SUM}
        self.series_dict = series_dict

    def get_values_from_series(self, series_tuple):
        series_ids, indexes, _ = series_tuple
        series_values = self.series_dict[series_ids[0]]
        start_index, end_index = indexes
        return series_values[start_index:end_index]

    def SUM(self, series_tuple_string: str):
        series_tuple = ast.literal_eval(series_tuple_string)
        return sum(self.get_values_from_series(series_tuple))

    def evaluate_sum(self, sum_formula: str):
        tree = ast.parse(sum_formula, mode="eval")
        compiled = compile(tree, filename="<ast>", mode="eval")
        result = eval(compiled, {"__builtins__": {}}, self.local_env)
        return result

class FormulaListGenerator:
    def __init__(self, formula_ast):
        self.formula_ast = formula_ast

    def adjust_indices(self, node, index_increment):
        if isinstance(node, ast_nodes.FunctionNode):
            self.adjust_function_node_indices(node, index_increment)
        elif hasattr(node, 'left') and hasattr(node, 'right'):
            self.adjust_indices(node.left, index_increment)
            self.adjust_indices(node.right, index_increment)

    def adjust_function_node_indices(self, node, index_increment):
        new_args = []
        for arg in node.args:
            new_args.append(self.adjust_operand_node(arg, index_increment) if isinstance(arg, ast_nodes.OperandNode) else arg)
        node.args = new_args

    def adjust_operand_node(self, arg, index_increment):
        parts = ast.literal_eval(arg.tvalue)
        series_tuple, indexes, deltas = parts
        updated_indexes = (indexes[0] + index_increment, indexes[1] + index_increment)
        updated_tvalue = f"(({repr(series_tuple[0])},), {updated_indexes}, {deltas})"
        return ast_nodes.OperandNode(tokenizer.f_token(tvalue=updated_tvalue, ttype="operand", tsubtype="text"))

    def generate_formulas_list(self, start_index: int, end_index: int):
        return [self.generate_single_formula(i) for i in range(start_index, end_index + 1)]

    def generate_single_formula(self, index_increment):
        new_ast = copy.deepcopy(self.formula_ast)
        self.adjust_indices(new_ast, index_increment)
        return new_ast


In [57]:
import xlcalculator.ast_nodes as ast_nodes
import xlcalculator.tokenizer as tokenizer

# Define the formula
formula = '(SUM("((\'Sheet1|col_1|2|2\',), (0, 1), (1, 1))")) + (SUM("((\'Sheet1|col_2|2|3\',), (0, 1), (1, 1))"))'

formula_ast = ast_nodes.OperatorNode(tokenizer.f_token(
    tvalue='+', ttype='operator-infix', tsubtype=''
))

formula_ast.left = ast_nodes.FunctionNode(tokenizer.f_token(
    tvalue="SUM", ttype="function", tsubtype=""
))

formula_ast.left.args = [
    ast_nodes.OperandNode(tokenizer.f_token(
        tvalue="(('Sheet1|col_1|2|2',), (0, 1), (1, 1))", ttype="operand", tsubtype="text"
    ))
]

formula_ast.right = ast_nodes.FunctionNode(tokenizer.f_token(
    tvalue="SUM", ttype="function", tsubtype=""
))

formula_ast.right.args = [
    ast_nodes.OperandNode(tokenizer.f_token(
        tvalue="(('Sheet1|col_2|2|3',), (0, 1), (1, 1))", ttype="operand", tsubtype="text"
    ))
]

In [58]:
# Instantiate the formula list generator with the created AST
generator = FormulaListGenerator(formula_ast)

# Generate a list of formulas adjusting indices from 0 to 2
formula_list = generator.generate_formulas_list(0, 2)

# The formula list now contains ASTs for SUM operation with adjusted indices


In [59]:
# Instantiate the formula evaluator
evaluator = FormulaEvaluator(# Series dictionary for example data
series_dict = {
    "Sheet1|col_1|2|2": pd.Series([1, 2, 3, 4, 5]),
    "Sheet1|col_2|2|3": pd.Series([6, 7, 8, 9, 10])
})


result = evaluator.evaluate_sum(str(formula_list[0]))

In [60]:
result

7